In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import glob
import os
from math import log

In [3]:
allFiles = sorted(glob.glob("*.txt"),key=os.path.getmtime) # includes all files into a single stucture,
# Sorted by modification time so that the first measurement is first...
print(allFiles)

['PMT1_Signal_280Deg_07182017.txt', 'PMT1_Background_280Deg_07182017.txt', 'PMT1_Background_275Deg_07182017.txt', 'PMT1_Signal_275Deg_07182017.txt', 'PMT1_Signal_270Deg_07182017.txt', 'PMT1_Background_270Deg_07182017.txt', 'PMT1_Background_265Deg_07182017.txt', 'PMT1_Signal_265Deg_07182017.txt', 'PMT1_Signal_260Deg_07182017.txt', 'PMT1_Background_260Deg_07182017.txt', 'PMT2_Signal_280Deg_07182017.txt', 'PMT2_Background_280Deg_07182017.txt', 'PMT2_Background_275Deg_07182017.txt', 'PMT2_Signal_275Deg_07182017.txt', 'PMT2_Signal_270Deg_07182017.txt', 'PMT2_Background_270Deg_07182017.txt', 'PMT2_Background_265Deg_07182017.txt', 'PMT2_Signal_265Deg_07182017.txt', 'PMT2_Signal_260Deg_07182017.txt', 'PMT2_Background_260Deg_07182017.txt', 'PMT3_Signal_280Deg_07182017.txt', 'PMT3_Background_280Deg_07182017.txt', 'PMT3_Background_275Deg_07182017.txt', 'PMT3_Signal_275Deg_07182017.txt', 'PMT3_Signal_270Deg_07182017.txt', 'PMT3_Background_270Deg_07182017.txt', 'PMT3_Background_265Deg_07182017.txt'

In [4]:
frame = pd.DataFrame()
df = pd.DataFrame()

In [5]:
for i,file_ in enumerate(allFiles):
#    f = open(file_,'r')
#     for j,line in enumerate(f):
#         if j == 1:
#             date = line[14:22]
#             break
    df = pd.read_csv(file_,index_col = None, header=None, skiprows = 10, skipfooter = 1, delimiter = '\t')
#     frame[file_] = df[5]
    df['file']=file_
    df['timeDelay']= df[1]
    df['PDvalue'] = df[3]
#     df['date']=date
    if i < 1:
        frame = df
    else:
        frame = frame.append(df)

/Users/sanjaykhatri/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.


In [6]:
c = 3*10**11; #Speed of Light in mm/s, Going to use this to convert spatial information to time. 
T1 = (718*0.000125 + 0.0229)*2*25.4/(10**-12*c)
def ConvertToTimeDelay(x,itr,offset):
    c = 3*10**11
    timeDelay = []
    for i in range(0,len(x)):
        timedelay = -((x[i,0]*itr + offset)*25.4*2/(10**-12*c))+T1
        timeDelay.append(timedelay)
    return(timeDelay)

In [44]:
times = pd.Series()
PDvalues = pd.Series()
AvgPD50 = pd.Series()
itr = 0.000125
offset = 0.023
for i in range(0,100):
    time = frame[1][i]
    PDvalue = frame[3][i]
    times = times.append(time)
    PDvalues = PDvalues.append(PDvalue)
#print(len(PDvalues))
PDvalues = PDvalues.values.reshape(100,len(allFiles))
Background = np.array([PDvalues[:,1],PDvalues[:,2],PDvalues[:,5],PDvalues[:,6],PDvalues[:,9],\
                       PDvalues[:,11],PDvalues[:,12],PDvalues[:,15],PDvalues[:,16],PDvalues[:,19],\
                       PDvalues[:,21],PDvalues[:,22],PDvalues[:,25],PDvalues[:,26],PDvalues[:,29],\
                       PDvalues[:,31],PDvalues[:,32],PDvalues[:,35],PDvalues[:,36],PDvalues[:,39],\
                       PDvalues[:,41],PDvalues[:,42],PDvalues[:,45],PDvalues[:,46],PDvalues[:,49]])
Background_mean = np.mean(Background, axis = 1)
PMT_Raw = np.array([PDvalues[:,0],PDvalues[:,3],PDvalues[:,4],PDvalues[:,7],PDvalues[:,8],\
                    PDvalues[:,10],PDvalues[:,13],PDvalues[:,14],PDvalues[:,17],PDvalues[:,18],\
                    PDvalues[:,20],PDvalues[:,23],PDvalues[:,24],PDvalues[:,27],PDvalues[:,28],\
                    PDvalues[:,30],PDvalues[:,33],PDvalues[:,34],PDvalues[:,37],PDvalues[:,38],\
                    PDvalues[:,40],PDvalues[:,43],PDvalues[:,44],PDvalues[:,47],PDvalues[:,48]])
MeanPMT1 = np.mean(PMT_Raw,axis = 1)

PMT_Net = MeanPMT1 - Background_mean
MeanNET = np.reshape(PMT_Net,(5,5)) # As you add more files, change the 4 to the number of runs. 7 is b/c I took 7 different angles
MeanNET2 = np.mean(MeanNET,axis =0)
THzEnergy = np.array([6.5,5.75,4.5,3.5,2.5])
PMT_STD = np.std(MeanNET,axis = 0)
PMT_Err = PMT_STD/(4**0.5)
w = (PMT_Err/(MeanNET2*log(10)))**-2

print(Background_mean)
print(MeanPMT1)
print(MeanNET)
#print(PMT_)
print(MeanNET2)
LogPMT = np.log10(np.absolute(MeanNET2))
LogTHz = np.log10(np.absolute(THzEnergy))
print(LogPMT)

[-0.584493 -0.572971 -0.553112 -0.540971 -0.537347 -0.581132 -0.563784
 -0.554387 -0.53734  -0.521525 -0.558071 -0.555454 -0.528772 -0.529063
 -0.513753 -0.566071 -0.555339 -0.540926 -0.523762 -0.510089 -0.549653
 -0.539662 -0.521926 -0.5      -0.506126]
[-0.727476 -0.697391 -0.651952 -0.591425 -0.560134 -0.692385 -0.674309
 -0.632946 -0.587493 -0.550378 -0.678621 -0.653515 -0.617961 -0.578622
 -0.548524 -0.673358 -0.647572 -0.613199 -0.566748 -0.536065 -0.66146
 -0.633442 -0.599327 -0.560694 -0.527376]
[[-0.142983 -0.12442  -0.09884  -0.050454 -0.022787]
 [-0.111253 -0.110525 -0.078559 -0.050153 -0.028853]
 [-0.12055  -0.098061 -0.089189 -0.049559 -0.034771]
 [-0.107287 -0.092233 -0.072273 -0.042986 -0.025976]
 [-0.111807 -0.09378  -0.077401 -0.060694 -0.02125 ]]
[-0.118776  -0.1038038 -0.0832524 -0.0507692 -0.0267274]
[-0.9252713  -0.98378675 -1.07960324 -1.29439968 -1.57304329]


In [45]:
Delta = sum(w)*sum(w*LogTHz**2) - (sum(w*LogTHz))**2
A = (sum(w*LogTHz**2)*sum(w*LogPMT) - sum(w*LogTHz)*sum(w*LogTHz*LogPMT))/Delta
B = (sum(w)*sum(w*LogTHz*LogPMT) - (sum(w*LogPMT)*sum(w*LogTHz)))/Delta
# Least Squares Fitting Parameters above
print(w)
print(A)
print(B)
err = (((len(LogPMT)-2)**-1) * sum((LogPMT - A -B*LogTHz)**2)**0.5)
errorA = (sum(w*LogTHz**2)/Delta)**0.5
errorB = (sum(w)/Delta)**0.5
print(errorA)
print(errorB)

[ 1810.45866398  1549.17736473  1615.15531632  1697.35531609   656.90759442]
-2.10781603008
1.48816546249
0.0607069278779
0.0893131824604


In [35]:
plt.plot(LogTHz, A + B*LogTHz,'r',label = 'Fit: y=1.49x -2.11')
plt.plot(LogTHz, LogPMT,'b.', label = 'Data')
plt.grid(True)
plt.xlabel('Log(Laser Power)')
plt.ylabel('Log(PMT Signal)')
plt.legend(loc = 'upper left')
plt.show()
#plt.savefig('LogPMT_Vs_LogTHz_07182017.png')